# Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import scipy as sp
from sklearn import metrics
import pandas
import os


In [ ]:
import math
from plotnine import *
from sklearn.linear_model import LogisticRegression
from lmfit import Parameters, Parameter, report_fit, Minimizer
from lmfit import minimize, fit_report,conf_interval, ci_report

In [ ]:
from Fitter import *

In [ ]:
from Vuong import *

# Load and explore data

## Load

In [ ]:
code_dir = os.getcwd()

In [ ]:
data_dir = os.path.split(code_dir)[0] + '\\Data\\'

In [ ]:
source_file = data_dir + "diabetes-dataset.csv"
diabetes = pandas.read_csv(source_file)

In [ ]:
diabetes = diabetes.reset_index()
diabetes['PatNR'] = 'Pat' + diabetes['index'].astype(str) 
diabetes = diabetes.drop(['index'], axis=1)

In [ ]:
diabetes.head()

## Explore

In [ ]:
%matplotlib inline

p = (ggplot(diabetes)         # defining what data to use
 + aes(x='BMI', y='Outcome')    # defining what variable to use
#  + geom_bar(size=20) # defining the type of plot to use
+ geom_point()  
+ theme_minimal()  


+labs(x="BMI", y = "Diabetes")
)
p

In [ ]:
p = (ggplot(diabetes)         # defining what data to use
 + aes(x='BloodPressure', y='Outcome')    # defining what variable to use
#  + geom_bar(size=20) # defining the type of plot to use
+ geom_point()  
+ theme_minimal()
+ labs(x="Blood Pressure", y = "Diabetes"))
p

# Fits

## lmfit

### Fit I - NTCP with negative log likelihood

#### Fit

In [ ]:
# print("start: now =", datetime.now().strftime("%Y%m%d-%H:%M"))

# initializations --------------------------------------------------------
# parameters -----------------------------------------------------------

method_global = "ampgo"
method_local = "L-BFGS-B"


model_df1 = pandas.DataFrame()

for clinical_parameter in ['BloodPressure', 'BMI']:
    # data ------------------------------------------
    diabetes_df = diabetes[['Outcome', clinical_parameter]]
    diabetes_df = diabetes_df.rename(columns={'Outcome': 'outcomes', clinical_parameter: 'values'})

    # fit ------------------------------------------
    params_population = Parameters()
    params_population.add('TD50',   value = 150, min = 0.0001, vary=True)  # min=0 Prevent k from becoming negative
    params_population.add('m',  value = 0.1, min = 0.0001, vary=True)


    fitter=Fit(minimizeNLL, params_population, diabetes_df, method_global, method_local)
    result_parameters1 = fitter.fit_data()
   
    # save ------------------------------------------
    for param in list(result_parameters1.params):
        val = result_parameters1.params[param].value
        dict_param = {'clinical_parameter': clinical_parameter,
                      'parameter': param,
                      'optimized_value': val,
                      'AIC': result_parameters1.aic}
        model_df1 = pd.concat([model_df1,pd.DataFrame([dict_param])], ignore_index=True)


In [ ]:
model_df1

#### Plot

In [ ]:
diabetes_df_plot = diabetes[['PatNR','Outcome', 'BloodPressure', 'BMI']]

In [ ]:
TD50_BP = model_df1.loc[(model_df1['clinical_parameter'] == 'BloodPressure') & (model_df1['parameter'] == 'TD50') ].iloc[0]['optimized_value']
m_BP = model_df1.loc[(model_df1['clinical_parameter'] == 'BloodPressure') & (model_df1['parameter'] == 'm') ].iloc[0]['optimized_value']


TD50_BMI = model_df1.loc[(model_df1['clinical_parameter'] == 'BMI') & (model_df1['parameter'] == 'TD50') ].iloc[0]['optimized_value']
m_BMI = model_df1.loc[(model_df1['clinical_parameter'] == 'BMI') & (model_df1['parameter'] == 'm') ].iloc[0]['optimized_value']

In [ ]:
diabetes_df_plot['Probability_BP'] =  diabetes_df_plot.apply(lambda row: CalcNTCP(row['BloodPressure'],TD50_BP,m_BP),axis=1)
diabetes_df_plot['Probability_BMI'] =  diabetes_df_plot.apply(lambda row: CalcNTCP(row['BMI'],TD50_BMI,m_BMI),axis=1)

In [ ]:
diabetes_df_plot.head()

In [ ]:
%matplotlib inline

plot_models1a = (ggplot(diabetes_df_plot)   
 + aes(x='BMI')    
+ geom_point(aes(y='Probability_BMI'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="BMI", y = "Probability of Diabetes")
)
plot_models1a

In [ ]:
%matplotlib inline

plot_models1b = (ggplot(diabetes_df_plot)   
 + aes(x='BloodPressure')    
+ geom_point(aes(y='Probability_BP'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="Blood Pressure", y = "Probability of Diabetes")
)
plot_models1b

#### Test significance

In [ ]:
diabetes_df_plot.head()

In [ ]:
BloodPressuredf = diabetes_df_plot[['PatNR','Outcome','BloodPressure', 'Probability_BP']]
BloodPressuredf.rename(columns={'Outcome': 'y', 'Probability_BP': 'y_pred'}, inplace=True)

In [ ]:
BMIdf = diabetes_df_plot[['PatNR','Outcome','BMI', 'Probability_BMI']]
BMIdf.rename(columns={'Outcome': 'y', 'Probability_BMI': 'y_pred'}, inplace=True)

In [ ]:
vuong(BloodPressuredf, BMIdf, dof1 = 2, dof2 = 2)

### Fit II - logistic regression with negative log likelihood

#### Fit

In [ ]:
# print("start: now =", datetime.now().strftime("%Y%m%d-%H:%M"))

# initializations --------------------------------------------------------
# parameters -----------------------------------------------------------



method_global = "ampgo"
method_local = "L-BFGS-B"
model_df2= pandas.DataFrame()

# patients ------------------------------------------
for clinical_parameter in ['BloodPressure', 'BMI']:
    # data ------------------------------------------
    diabetes_df = diabetes[['Outcome', clinical_parameter]]
    diabetes_df = diabetes_df.rename(columns={'Outcome': 'outcomes', clinical_parameter: 'values'})

    # fit ------------------------------------------
    params_population = Parameters()

    params_population.add('b0',   value = -5,  vary=True)  # min=0 Prevent k from becoming negative
    params_population.add('b1',  value = 0.03,  vary=True)


    fitter=Fit(logistic_regression_NLL, params_population, diabetes_df, method_global, method_local)
    result_parameters2 = fitter.fit_data()
    # save ------------------------------------------
    for param in list(result_parameters2.params):
        val = result_parameters2.params[param].value
        dict_param = {'clinical_parameter': clinical_parameter,
                      'parameter': param,
                      'optimized_value': val,
                      'AIC': result_parameters2.aic}
        
        model_df2 = pd.concat([model_df2,pd.DataFrame([dict_param])], ignore_index=True)


In [ ]:
model_df2

#### Plot

In [ ]:
diabetes_df_plot = diabetes[['Outcome', 'BloodPressure', 'BMI']]

In [ ]:
b0_BP = model_df2.loc[(model_df2['clinical_parameter'] == 'BloodPressure') & (model_df2['parameter'] == 'b0') ].iloc[0]['optimized_value']
b1_BP = model_df2.loc[(model_df2['clinical_parameter'] == 'BloodPressure') & (model_df2['parameter'] == 'b1') ].iloc[0]['optimized_value']


b0_BMI = model_df2.loc[(model_df2['clinical_parameter'] == 'BMI') & (model_df2['parameter'] == 'b0') ].iloc[0]['optimized_value']
b1_BMI = model_df2.loc[(model_df2['clinical_parameter'] == 'BMI') & (model_df2['parameter'] == 'b1') ].iloc[0]['optimized_value']

In [ ]:
diabetes_df_plot['Probability_BP'] =  diabetes_df_plot.apply(lambda row: logistic_regression(row['BloodPressure'], b0_BP, b1_BP), axis=1)
diabetes_df_plot['Probability_BMI'] =  diabetes_df_plot.apply(lambda row: logistic_regression(row['BMI'], b0_BMI, b1_BMI),axis=1)

In [ ]:
diabetes_df_plot.head()

In [ ]:
%matplotlib inline

plot_models2a = (ggplot(diabetes_df_plot)   
 + aes(x='BloodPressure')    
+ geom_point(aes(y='Probability_BP'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="Blood Pressure", y = "Probability of Diabetes")
)
plot_models2a

In [ ]:
%matplotlib inline

plot_models2b = (ggplot(diabetes_df_plot)   
 + aes(x='BMI')    
+ geom_point(aes(y='Probability_BMI'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="BMI", y = "Probability of Diabetes")
)
plot_models2b

### Fit III logistic regression - with least squares

#### Fit

In [ ]:
# print("start: now =", datetime.now().strftime("%Y%m%d-%H:%M"))

# initializations --------------------------------------------------------
# parameters -----------------------------------------------------------

method_global = "leastsq"
method_local = None

model_df3 = pandas.DataFrame()
for clinical_parameter in ['BloodPressure', 'BMI']:
    # data ------------------------------------------
    diabetes_df = diabetes[['Outcome', clinical_parameter]]
    diabetes_df = diabetes_df.rename(columns={'Outcome': 'outcomes', clinical_parameter: 'values'})

    # fit ------------------------------------------
    params_population = Parameters()


    params_population.add('b0',   value = 1,  vary=True)  # min=0 Prevent k from becoming negative
    params_population.add('b1',  value = 0,  vary=True)


    fitter=Fit(logistic_regression_residuals, params_population, diabetes_df, method_global, method_local)
    result_parameters3 = fitter.fit_data()
    # save ------------------------------------------
    for param in list(result_parameters3.params):
        val = result_parameters3.params[param].value
        dict_param = {'clinical_parameter': clinical_parameter,
                      'parameter': param,
                      'optimized_value': val,
                      'AIC': result_parameters3.aic}
        
        model_df3 = pd.concat([model_df3,pd.DataFrame([dict_param])], ignore_index=True)

In [ ]:
model_df3

#### plot

In [ ]:
diabetes_df_plot = diabetes[['Outcome', 'BloodPressure', 'BMI']]

In [ ]:
b0_BP = model_df3.loc[(model_df3['clinical_parameter'] == 'BloodPressure') & (model_df3['parameter'] == 'b0') ].iloc[0]['optimized_value']
b1_BP = model_df3.loc[(model_df3['clinical_parameter'] == 'BloodPressure') & (model_df3['parameter'] == 'b1') ].iloc[0]['optimized_value']


b0_BMI = model_df3.loc[(model_df3['clinical_parameter'] == 'BMI') & (model_df3['parameter'] == 'b0') ].iloc[0]['optimized_value']
b1_BMI = model_df3.loc[(model_df3['clinical_parameter'] == 'BMI') & (model_df3['parameter'] == 'b1') ].iloc[0]['optimized_value']

In [ ]:
diabetes_df_plot['Probability_BP'] =  diabetes_df_plot.apply(lambda row: logistic_regression(row['BloodPressure'], b0_BP, b1_BP), axis=1)
diabetes_df_plot['Probability_BMI'] =  diabetes_df_plot.apply(lambda row: logistic_regression(row['BMI'], b0_BMI, b1_BMI),axis=1)

In [ ]:
diabetes_df_plot.head()

In [ ]:
%matplotlib inline

plot_models3a = (ggplot(diabetes_df_plot)   
 + aes(x='BloodPressure')    
+ geom_point(aes(y='Probability_BP'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="Blood Pressure", y = "Probability of Diabetes")
)
plot_models3a

In [ ]:
%matplotlib inline

plot_models3b = (ggplot(diabetes_df_plot)   
 + aes(x='BMI')    
+ geom_point(aes(y='Probability_BMI'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="BMI", y = "Probability of Diabetes")
)
plot_models3b

## sckitlearn

### Fit

In [ ]:
feature_cols =  ['BloodPressure', 'BMI']

model_df4 = pandas.DataFrame()
for feature in feature_cols:
    X = diabetes.loc[:, [feature]]
    y = diabetes.Outcome
    
    model = LogisticRegression(solver ='lbfgs',max_iter=1000)
    model.fit(X, y)
    b1 = model.coef_[0,0]
    b0 = model.intercept_[0]
    
    
    # save ------------------------------------------
    for p in ['b0','b1']:
        if p=='b0': val=b0
        if p=='b1': val=b1
        dict_param = {'clinical_parameter': feature,
                      'parameter': p,
                      'optimized_value': val,
    #                   'AIC': result_parameters3.aic
                     }
        
        model_df4 = pd.concat([model_df4,pd.DataFrame([dict_param])], ignore_index=True)

In [ ]:
model_df4

### Plot

In [ ]:
diabetes_df_plot = diabetes[['Outcome', 'BloodPressure', 'BMI']]

In [ ]:
b0_BP = model_df4.loc[(model_df4['clinical_parameter'] == 'BloodPressure') & (model_df4['parameter'] == 'b0') ].iloc[0]['optimized_value']
b1_BP = model_df4.loc[(model_df2['clinical_parameter'] == 'BloodPressure') & (model_df4['parameter'] == 'b1') ].iloc[0]['optimized_value']


b0_BMI = model_df4.loc[(model_df4['clinical_parameter'] == 'BMI') & (model_df4['parameter'] == 'b0') ].iloc[0]['optimized_value']
b1_BMI = model_df4.loc[(model_df4['clinical_parameter'] == 'BMI') & (model_df4['parameter'] == 'b1') ].iloc[0]['optimized_value']

In [ ]:
diabetes_df_plot['Probability_BP'] =  diabetes_df_plot.apply(lambda row: logistic_regression(row['BloodPressure'], b0_BP, b1_BP), axis=1)
diabetes_df_plot['Probability_BMI'] =  diabetes_df_plot.apply(lambda row: logistic_regression(row['BMI'], b0_BMI, b1_BMI),axis=1)

In [ ]:
diabetes_df_plot.head()

In [ ]:
%matplotlib inline

plot_models2a = (ggplot(diabetes_df_plot)   
 + aes(x='BloodPressure')    
+ geom_point(aes(y='Probability_BP'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="Blood Pressure", y = "Probability of Diabetes")
)
plot_models2a

In [ ]:
%matplotlib inline

plot_models2b = (ggplot(diabetes_df_plot)   
 + aes(x='BMI')    
+ geom_point(aes(y='Probability_BMI'), colour = 'red')
+ geom_point(aes(y='Outcome'), alpha = 0.05)
+ theme_minimal() 
+labs(x="BMI", y = "Probability of Diabetes")
)
plot_models2b